In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("../")
from utils.helper_function import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

import optuna
import pickle
import datetime

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=UserWarning)

/tmp/ipykernel_21474/3859719561.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_sk = pd.read_csv("../data/eye-irritation/train.csv")
test_sk = pd.read_csv("../data/eye-irritation/val.csv")

In [3]:
train_sk.head()

,SMILES,Activity
0,CCOP(OC1=C(C=C(C=C1C)C(C)(C)C)C(C)(C)C)OC1=C(C...,0
1,CCCCCCCCCCCCCCCCSC1NC2=CC(=CC=C2N=1)S(O)(=O)=O,1
2,O=C(CC(=O)CC1=CC(F)=C(F)C=C1F)N1CC2=NN=C(N2CC1...,0
3,CC1C=C(N)N(N=1)C1C=CC=CC=1,0
4,CC1CCCC(C)(C)C=1C(=O)C=CC,0


In [4]:
test_sk.head()

,SMILES,Activity
0,COC1=CC=C(CC2CC2)C=C1,1
1,CC(C)OP(=O)(OC(C)C)SCC1C=CC=CC=1,1
2,NNC(N)=O,1
3,OS(=O)(=O)C1C=CC=C2C=C(C=C(NC3C=CC=CC=3)C2=1)N...,1
4,CCCC1COC(CC1)C1C=CC(O)=CC=1,1


In [5]:
smilesTrain = train_sk["SMILES"].to_numpy()
activityTrain = train_sk["Activity"].to_numpy()

smilesTest = test_sk["SMILES"].to_numpy()
activityTest = test_sk["Activity"].to_numpy()

In [6]:
idx = valid_indices(smilesTrain)
idx2 = valid_indices(smilesTest)

smilesTrain = smilesTrain[idx]
smilesTest = smilesTest[idx2]

[14:27:29] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:27:29] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 18 19 20 21 22 23
[14:27:29] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18
[14:27:29] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18
[14:27:29] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 15 16 17 19 20 21 25 27 28
[14:27:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15


In [7]:
maccs_train = maccs_fp(smilesTrain)
maccs_test = maccs_fp(smilesTest)


In [8]:
activityTrain = activityTrain[idx]
activityTest = activityTest[idx2]

In [9]:
maccs_train = np.array(maccs_train) 
maccs_test = np.array(maccs_test)



In [10]:
def svm_modl(trial, xtrain, ytrain):
    n = trial.suggest_int('degree', 2, 5)
    model_svm = SVC(degree= n)

    scores = cross_validate(model_svm, xtrain, ytrain, cv=5, scoring='roc_auc')
    mean_roc = scores['test_score'].mean()

    return 1/(mean_roc + 1e-6)

In [18]:
study2 = optuna.create_study(study_name='SVM_study_eye_irritation_maccs', storage= None)  # Create a new study.
study2.optimize(lambda trial: svm_modl(trial, maccs_train, activityTrain), n_trials=20)  # Invoke optimization of the objective function.

[I 2024-01-31 14:29:38,961] A new study created in memory with name: SVM_study_eye_irritation
[I 2024-01-31 14:29:43,476] Trial 0 finished with value: 1.257324474116107 and parameters: {'degree': 5}. Best is trial 0 with value: 1.257324474116107.
[I 2024-01-31 14:29:47,910] Trial 1 finished with value: 1.257324474116107 and parameters: {'degree': 3}. Best is trial 0 with value: 1.257324474116107.
[I 2024-01-31 14:29:52,764] Trial 2 finished with value: 1.257324474116107 and parameters: {'degree': 2}. Best is trial 0 with value: 1.257324474116107.
[I 2024-01-31 14:29:57,844] Trial 3 finished with value: 1.257324474116107 and parameters: {'degree': 4}. Best is trial 0 with value: 1.257324474116107.
[I 2024-01-31 14:30:03,822] Trial 4 finished with value: 1.257324474116107 and parameters: {'degree': 4}. Best is trial 0 with value: 1.257324474116107.
[I 2024-01-31 14:30:08,218] Trial 5 finished with value: 1.257324474116107 and parameters: {'degree': 2}. Best is trial 0 with value: 1.25732

In [19]:
best_model2 = SVC(degree= study2.best_params['degree'])
best_model2.fit(maccs_train, activityTrain)

SVC(degree=5)

In [20]:
y_pred = best_model2.predict(maccs_test)
accuracy2 = accuracy_score(y_true= activityTest, y_pred= y_pred)
print(accuracy2)

0.7741935483870968


In [21]:
tn, fp, fn, tp = confusion_matrix(y_pred, activityTest).ravel()

ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')


Accuracy = 0.7741935483870968
Sensitivity = 0.7867528271405493
Specificity = 0.7243589743589743


In [15]:
model_name = f"SVM_MACCS_model_Eye_Irritaion{datetime.datetime.now().isoformat()}"
model_path = "../models/"
file = os.path.join(model_path, model_name)
pickle.dump(best_model2, open(file, 'wb'))